## Download data from NYC TLC Trip Record Data

https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page

In [1]:
%pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9673 sha256=fd5a4728e530490b5dff80b996f02ef67b24f0948b6d336ac9a3f6faa8043db3
  Stored in directory: /root/.cache/pip/wheels/ba/78/fb/e0c24a9e73d7483b073d15b7e05f43f3fc2ac75eff6899c7aa
Successfully built wget
Note: you may need to restart the kernel to use updated packages.


In [2]:
import wget
import os
import shutil
path = "/data/"

urls = [
    'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet',
    'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet',
    'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-03.parquet'
]

for url in urls:
    if os.path.exists(os.path.join(path, url.split("/")[-1])):
        continue
    print(f"Downloading {url}...")
    file_name = wget.download(url)
    shutil.move(file_name, os.path.join(path,  file_name))  

## Auto reload libraries

In [3]:
%load_ext autoreload
%autoreload 2

## Initialise spark

In [4]:
from flypipe.tests.spark import spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/22 04:02:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Create database

In [5]:
spark.sql("drop database if exists raw_nyc_trip cascade")
spark.sql("create database raw_nyc_trip")
spark.sql("show databases")

namespace
default
raw_nyc_trip


## Load data into PySpark Dataframe

In [6]:
df=spark.read.parquet("/data/*.parquet")

## Save data into data warehouse

In [7]:
import shutil
spark.sql("drop table if exists raw_nyc_trip.yellow_trip")
shutil.rmtree("/spark-warehouse/raw_nyc_trip.db/yellow_trip", ignore_errors = True)
df.write.mode("overwrite").saveAsTable("raw_nyc_trip.yellow_trip")

22/11/22 04:03:41 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


In [8]:
spark.sql("select * from raw_nyc_trip.yellow_trip")

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
1,2022-03-01 00:13:08,2022-03-01 00:24:35,1.0,2.4,1.0,N,90,209,2,10.0,3.0,0.5,0.0,0.0,0.3,13.8,2.5,0.0
1,2022-03-01 00:47:52,2022-03-01 01:00:08,1.0,2.2,1.0,N,148,234,2,10.5,3.0,0.5,0.0,0.0,0.3,14.3,2.5,0.0
2,2022-03-01 00:02:46,2022-03-01 00:46:43,1.0,19.78,2.0,N,132,249,1,52.0,0.0,0.5,11.06,0.0,0.3,67.61,2.5,1.25
2,2022-03-01 00:52:43,2022-03-01 01:03:40,2.0,2.94,1.0,N,211,66,1,11.0,0.5,0.5,4.44,0.0,0.3,19.24,2.5,0.0
2,2022-03-01 00:15:35,2022-03-01 00:34:13,1.0,8.57,1.0,N,138,197,1,25.0,0.5,0.5,5.51,0.0,0.3,33.06,0.0,1.25
1,2022-03-01 00:11:57,2022-03-01 00:53:05,2.0,14.0,1.0,N,132,33,1,43.5,1.75,0.5,9.2,0.0,0.3,55.25,0.0,1.25
2,2022-03-01 00:05:11,2022-03-01 00:08:22,1.0,0.61,1.0,N,166,151,1,4.5,0.5,0.5,1.0,0.0,0.3,6.8,0.0,0.0
2,2022-03-01 00:30:56,2022-03-01 00:46:21,1.0,2.83,1.0,N,74,238,1,13.0,0.5,0.5,3.7,0.0,0.3,18.0,0.0,0.0
2,2022-03-01 00:30:28,2022-03-01 00:30:36,1.0,0.1,1.0,N,145,145,3,-2.5,-0.5,-0.5,0.0,0.0,-0.3,-3.8,0.0,0.0
2,2022-03-01 00:30:28,2022-03-01 00:30:36,1.0,0.1,1.0,N,145,145,2,2.5,0.5,0.5,0.0,0.0,0.3,3.8,0.0,0.0
